# 信用卡詐欺偵測 - 挑戰二
融合 Isolation Forest 與 XGBoost 提升預測效果

In [ ]:
# 匯入必要套件
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

## 載入與預處理資料

In [ ]:
# 請先上傳 creditcard.csv
data = pd.read_csv('creditcard.csv')
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
data = data.drop(['Time'], axis=1)
data['Class'] = data['Class'].astype(int)
X = data.drop('Class', axis=1)
y = data['Class']

## Isolation Forest：生成異常分數

In [ ]:
iso_forest = IsolationForest(n_estimators=100, contamination=0.0015, random_state=42)
iso_forest.fit(X)
# 異常分數越小越可疑
scores = -iso_forest.decision_function(X)
# 將異常分數加入特徵中
X['anomaly_score'] = scores

## 分割資料並使用 XGBoost 訓練

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))